# Import datasets

In [1]:
import pandas as pd

In [2]:
popularity = pd.read_csv("steamcharts.csv")
appdata = pd.read_csv("steam_app_data.csv", dtype={10: "string"})
headers = pd.read_csv("id_name.csv")

# Keep only rows that we have player data for

**Part of the cleaning process, does nothing the second time around**

In [3]:
usable_ids = popularity['steam_appid'].unique()

In [4]:
appdata = appdata[appdata['steam_appid'].isin(usable_ids)]
headers = headers[headers['appid'].isin(usable_ids)]

## Write the changes back to file - save space

In [5]:
appdata.to_csv('steam_app_data.csv', index=False)
headers.to_csv('id_name.csv', index=False)

In [ ]:
appdata.columns

In [ ]:
popularity.columns

In [6]:
#Removing columns that will not be of use
appdata = appdata.drop(['name','dlc', 'detailed_description', 'about_the_game','short_description',
                        'fullgame','header_image','website', 'pc_requirements', 'mac_requirements','linux_requirements','legal_notice',
                        'drm_notice', 'ext_user_account_notice', 'demos', 'price_overview',
                        'packages', 'package_groups', 'reviews','screenshots','movies', 'achievements', 'controller_support', 'metacritic',
                        'support_info', 'background','content_descriptors', 'recommendations'], axis=1)
popularity = popularity.drop(['name', 'gain'],  axis=1)

In [ ]:
appdata.columns

In [7]:
#Changing data types
popularity['month'] = pd.to_datetime(popularity['month'], format='%b-%y')
#In case it interprets 25 as 1925 and not 2025
popularity.loc[popularity['month'].dt.year < 2000, 'month'] += pd.DateOffset(years=100)

## Merging datasets

In [ ]:
headers.head()

In [ ]:
popularity.head()

In [ ]:
appdata.head()

In [8]:
main = (
    popularity
        .merge(headers, left_on='steam_appid', right_on='appid', how='left')
        .merge(appdata, on='steam_appid', how='left')
)

main = main.drop(columns='appid')
main.head()



,month,avg_players,gain_percent,peak_players,steam_appid,name,type,required_age,is_free,supported_languages,developers,publishers,platforms,categories,genres,release_date
0,2025-09-01,7805.25,0.1276,13254,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}"
1,2025-08-01,6922.13,-0.0610,12168,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}"
2,2025-07-01,7371.48,-0.1016,13951,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}"
3,2025-06-01,8204.98,-0.0936,15798,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}"
4,2025-05-01,9052.51,-0.0495,15333,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}"


In [9]:
import re

def clean_languages(cell):
    if pd.isna(cell):
        return []
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', cell)
    # Remove extra notes like "*languages with full audio support"
    text = re.sub(r'\*.*languages with full audio support', '', text, flags=re.IGNORECASE)
    languages = [lang.strip() for lang in text.split(',') if lang.strip()]
    # Split by comma
    return languages

# Apply cleaning
main['supported_languages_clean'] = main['supported_languages'].apply(clean_languages)

# Flatten all lists to get unique languages
all_languages = [lang for sublist in main['supported_languages_clean'] for lang in sublist]
unique_languages = sorted(set(all_languages))

print(unique_languages)

['#lang_slovakian', 'Afrikaans', 'Albanian', 'Alemán', 'Alemán*', 'Alemão', 'Alemão*', 'Allemand', 'Allemand*', 'Amharic', 'Angielski', 'Angielski*', 'Anglais', 'Anglais*', 'Anglais**Langues avec support audio complet', 'Angol*', 'Arabic', 'Arabisch', 'Armenian', 'Assamese', 'Azerbaijani', 'Bangla', 'Basque', 'Belarusian', 'Bhs. Belanda', 'Bhs. Ceko*bahasa dengan dukungan audio penuh', 'Bhs. Inggris*', 'Bhs. Italia*', 'Bhs. Jepang*', 'Bhs. Jerman*', 'Bhs. Korea', 'Bhs. Polandia', 'Bhs. Portugis - Brasil*', 'Bhs. Prancis*', 'Bhs. Rusia*', 'Bhs. Spanyol - Spanyol*', 'Bhs. Tionghoa Sederhana', 'Bhs. Tionghoa Tradisional', 'Bosnian', 'Brasilianisches Portugiesisch', 'Brasilianisches Portugiesisch*', 'Brasilianisches Portugiesisch*Sprachen mit voller Audiounterstützung', 'Brazíliai portugál', 'Bulgarian', 'Bułgarski', 'Búlgaro', 'Catalan', 'Ceco*lingue con supporto audio completo', 'Checo', 'Checo*idiomas con localización de audio', 'Cherokee', 'Chinesisch (traditionell)', 'Chinesisch (trad

In [20]:
allowed_tags = ['English', 'Spanish', 'French', 'German', 'Russian', 'Chinese', 'Japanese', 'Italian', 'Finnish', 'Estonian', 'Swedish']

# Filter rows where 'tags' list contains any allowed_tags
main['supported_languages_clean'] = main['supported_languages_clean'].apply(
    lambda lst: [tag for tag in lst if tag in allowed_tags]
)

main.head()

,month,avg_players,gain_percent,peak_players,steam_appid,name,type,required_age,is_free,supported_languages,developers,publishers,platforms,categories,genres,release_date,supported_languages_clean
0,2025-09-01,7805.25,0.1276,13254,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}",[English]
1,2025-08-01,6922.13,-0.0610,12168,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}",[English]
2,2025-07-01,7371.48,-0.1016,13951,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}",[English]
3,2025-06-01,8204.98,-0.0936,15798,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}",[English]
4,2025-05-01,9052.51,-0.0495,15333,10,Counter-Strike,game,0.0,False,"English<strong>*</strong>, French<strong>*</st...",['Valve'],['Valve'],"{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","{'coming_soon': False, 'date': '1 Nov, 2000'}",[English]
